In [0]:
# !pip install spacy
# !pip install rasa_nlu
# !pip install rasa_core
# !pip install sklearn_crfsuite

In [0]:
intent_data="""
##intent: introduction
- how are you?
- howdy?
- h r u?
- hi
- hey
- hello

##intent: ask_time
- whats the time now
- can you tell me the time
- time now please?
- tell me the tim now

##intent: ask_weather
- whats the temperature in Bangalore now
- tell me the temperature in Bangalore now
- how is the weather condition in Bangalore
- can you tell me about the Bangalore weather condition

##intent: ask_hashtags
- whats are the trending hastags today
- today trending hastags please
- tell me what is trending today
- list top 10 trending hashtags
"""

In [0]:
%store intent_data > data.md

Writing 'intent_data' (str) to file 'data.md'.


In [0]:
import spacy
spacy.load('en')

In [0]:
setting="""
pipeline: spacy_sklearn
"""

%store setting > setting.yml

Writing 'setting' (str) to file 'setting.yml'.


In [0]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer,Interpreter

In [0]:
trainer=Trainer(config.load('setting.yml'))
training_data=load_data('data.md')
trainer.train(training_data)

ValueError: ignored

In [0]:
model_directory=trainer.persist('/models/',
                               fixed_model_name='current')
interpreter=Interpreter.load('/models/default/current')

In [0]:
interpreter.parse('h r u')

In [0]:
# for the reply
domain_text="""
intents:
-introduction
-ask_time
-ask_weather
-ask_hashtags

actions:
-utter_introduce
-utter_time
-utter_weather
-utter_hashtags

templates:
  utter_introduce:
  -Welcome
  -Hi, nice to meet you
  -hello, hope you are doing good
  utter_time:
  -Your time is really bad
  -Your time is really good
  utter_weather:
  -Its cloudy
  -its sunny
  -Why do you care
  utter_hashtags:
  -#IndvsAus,#WelcomeModi,#WelcomeAbhinandan
  -#Womansday
"""

%store domain_text > domain.yml

Writing 'domain_text' (str) to file 'domain.yml'.


In [0]:
stories="""
## introduce
*introduction
-utter_introduce

## Ask Time
*ask_time

## Ask Weather
*ask_weather
-utter_weather

## Ask hashtags
*ask_hastags
-utter_hastags
"""

%store stories > stories.md

Writing 'stories' (str) to file 'stories.md'.


In [0]:
from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

TypeError: ignored

In [0]:
domain_file='domain.yml'
model_path='model/dialogue'
training_data_file='stories.md'
agent=Agent(domain_file,
           policies=[MemoizationPolicy(max_history=3),KerasPolicy()])
training_data=agent.load_data(training_data_file)
agent.train(training_data)
agent.persist(model_path)